ASSIGNMENT 5 : Group #5
Adriana Sierra, Alejandra Navarro , Karla Chauca y Elisa Vivar

#### Importing libraries

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
from io import StringIO

#### The url to be scraped opens


In [7]:
driver = webdriver.Chrome(service=webdriver.chrome.service.Service(ChromeDriverManager().install()))
url='https://infogob.jne.gob.pe/Eleccion'
driver.get(url)
time.sleep(5)

#### First step: Tipo de proceso

In [8]:
#Select "Elecciones presidenciales" en "Tipo de proceso"
driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[2]/div[2]/div[1]/div').click()
driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[2]/div[2]/div[1]/div/div[2]/div[2]').click()

#### Second step: Elecciones

In [9]:
# Select: Elección
driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[2]/div[2]/div[2]/div').click()

# Get all the elections

## List of elections
elections_list = driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[2]/div[2]/div[2]/div/div[2]')

## Elections
elections = [ i for i in elections_list.find_elements(By.TAG_NAME, "div") ]

# Dictionary of the elections and their web elements
dict_elections = { election.text : election for election in elections }

#### Dataframe final

In [ ]:
df_resultados = pd.DataFrame(columns=['ELECCIONES', 'ORGANIZACIÓN POLÍTICA', 'TOTAL VOTOS'])

#### Loop to get all the elections' tables

In [ ]:
# Create a loop based on the dictionary
for x in list(dict_elections.keys())[1:]:
    elemento = dict_elections[x]
    elemento.click()

    # Select the election
    boton_ver_datos = WebDriverWait(driver, 3).until(
        EC.element_to_be_clickable((By.ID, 'btnVerDatos'))
    ).click()
    time.sleep(5)
    
    # Go to the section "Candidatos y resultados"
    driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[3]/div[1]/ul/li[2]').click()
    time.sleep(5)

    # Get the table for each election
    table_1= driver.find_element(By.XPATH,'/html/body/div[1]/section/div[2]/div[3]/div[3]/div/div/div/div[1]/div[2]/div[2]/div[2]/table')
    table1_html=table_1.get_attribute("outerHTML")
    html_io = StringIO(table1_html)
    table1= pd.read_html(html_io)[0]

    table1['ELECCIONES'] = x
    new_order1 = ['ELECCIONES', 'ORGANIZACIÓN POLÍTICA', 'TOTAL VOTOS']
    df1 = table1[new_order1]
    
    # Concat the new tables in the final dataframe
    df_resultados = pd.concat([df_resultados, df1], ignore_index=True)
    
    # Back to the beginning
    driver.back()
    time.sleep(5)

    driver.back()
    time.sleep(5)
    
    # On the list, to select a new election
    driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[2]/div[2]/div[2]/div').click()
    time.sleep(5)

driver.quit()


In [4]:
# Verify the final output
df_resultados

NameError: name 'df_resultados' is not defined

In [ ]:
# Consider
file_name = "resultados_elecciones.xlsx"
sheet = "Resultados"

# Export the final df
df_resultados.to_excel(file_name, sheet_name=sheet, index=False)

print(f"All the results of presidential elections are in this file: {file_name}")